In [1]:
import warnings
warnings.filterwarnings('ignore')    # 학습과정에서 발생하는 경고창을 무시하는 코드

## 1.문제정의
- 보스턴 주택 가격 데이터를 사용하여 주택 가격을 예측해보자
- 회귀모델
- LinearRegression, SGDRegressor

## 2. 데이터 수집

In [4]:
import pandas as pd
boston_data = pd.read_csv('boston_house.csv')
boston_data.head(1)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.09,1.0,296.0,15.3,396.9,4.98,24.0


In [5]:
X = boston_data.drop('MEDV', axis = True)   # 문제데이터
y = boston_data['MEDV']  # 답데이터

# 분리하고나면 항상 크기 확인하기
print(X.shape, y.shape)

(506, 13) (506,)


In [6]:
y.name = 'PRICE'

In [7]:
y

0      24.0
1      21.6
2      34.7
3      33.4
4      36.2
       ... 
501    22.4
502    20.6
503    23.9
504    22.0
505    11.9
Name: PRICE, Length: 506, dtype: float64

In [8]:
X.info()   # ✨문제데이터는 항상 숫자 형태로 생겨야한다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
dtypes: float64(13)
memory usage: 51.5 KB


## 3. 데이터전처리
- 보스턴 집값 데이터는 학습용 데이터이기 때문에 전처리가 필요없다.

## 4. 탐색적 데이터 분석(EDA)
### 상관관계 분석
- 컬럼간의 선형도(상관도)를 확인

In [11]:
boston_data.corr()    # correlation(상관관계)
# 상관계수(-1 ~ 1) : -1과 1에 가까울수록 두 컬럼이 관계가 깊다, 0에 가까울수록 관계가 없다.

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
CRIM,1.000000,-0.200469,0.406583,-0.055892,0.420972,-0.219247,0.352734,-0.379670,0.625505,0.582764,0.289946,-0.385064,0.455621,-0.388305
ZN,-0.200469,1.000000,-0.533828,-0.042697,-0.516604,0.311991,-0.569537,0.664408,-0.311948,-0.314563,-0.391679,0.175520,-0.412995,0.360445
INDUS,0.406583,-0.533828,1.000000,0.062938,0.763651,-0.391676,0.644779,-0.708027,0.595129,0.720760,0.383248,-0.356977,0.603800,-0.483725
CHAS,-0.055892,-0.042697,0.062938,1.000000,0.091203,0.091251,0.086518,-0.099176,-0.007368,-0.035587,-0.121515,0.048788,-0.053929,0.175260
NOX,0.420972,-0.516604,0.763651,0.091203,1.000000,-0.302188,0.731470,-0.769230,0.611441,0.668023,0.188933,-0.380051,0.590879,-0.427321
RM,-0.219247,0.311991,-0.391676,0.091251,-0.302188,1.000000,-0.240265,0.205246,-0.209847,-0.292048,-0.355501,0.128069,-0.613808,0.695360
AGE,0.352734,-0.569537,0.644779,0.086518,0.731470,-0.240265,1.000000,-0.747881,0.456022,0.506456,0.261515,-0.273534,0.602339,-0.376955
DIS,-0.379670,0.664408,-0.708027,-0.099176,-0.769230,0.205246,-0.747881,1.000000,-0.494588,-0.534432,-0.232471,0.291512,-0.496996,0.249929
RAD,0.625505,-0.311948,0.595129,-0.007368,0.611441,-0.209847,0.456022,-0.494588,1.000000,0.910228,0.464741,-0.444413,0.488676,-0.381626
TAX,0.582764,-0.314563,0.720760,-0.035587,0.668023,-0.292048,0.506456,-0.534432,0.910228,1.000000,0.460853,-0.441808,0.543993,-0.468536


| 영어 컬럼명  | 한글 번역                                      |
| ------- | ------------------------------------------ |
| CRIM    | 범죄율 (인구 1인당 도시 범죄율)                        |
| ZN      | 25,000 평방피트를 초과하는 주거지역 비율                  |
| INDUS   | 비소매 상업지역 면적 비율                             |
| CHAS    | 찰스강 인접 여부 (1: 인접, 0: 비인접)                  |
| NOX     | 대기 오염 농도 (산화질소 농도)                         |
| RM      | 주택당 평균 방 개수                                |
| AGE     | 1940년 이전에 건축된 주택 비율                        |
| DIS     | 5개 보스턴 고용센터까지의 거리 (가중치 포함 거리)              |
| RAD     | 방사형 고속도로 접근성 지수                            |
| TAX     | 재산세율 (10,000달러당 세금)                        |
| PTRATIO | 학생-교사 비율                                   |
| B       | 흑인 인구 비율 지수 (1000(Bk - 0.63)^2, Bk는 흑인 비율) |
| LSTAT   | 하위 계층 비율 (인구 중 사회경제적으로 낮은 비율)              |
| MEDV    | 주택 가격의 중앙값 (단위: \$1,000)                   |


## 5. 모델 선택 및 하이퍼파라미터 튜닝

In [14]:
# 수학적 공식을 이용한 해석적 모델(Linear Regression)
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
# 경사하강법
from sklearn.linear_model import SGDRegressor
sgd_model = SGDRegressor(eta0 = 0.01 , verbose = 1)

### 5-1. 훈련데이터와 평가데이터로 분리
- 학습지 7 : 시험지 3

In [16]:
# 랜덤샘플링 -> 전체 데이터를 7:3 비율로 섞어서 분류해주는 도구
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.3,  # 시험지의 비율
                                                    random_state = 5) # 랜덤 시드값

In [17]:
# 크기확인
print('문제 데이터 크기 :', X_train.shape, y_train.shape)
print('답 데이터 크기 :', X_test.shape, y_test.shape)

문제 데이터 크기 : (354, 13) (354,)
답 데이터 크기 : (152, 13) (152,)


## 6. 학습
- 학습에 필요한 데이터 : X_train, y_train

In [19]:
linear_model.fit(X_train, y_train)
sgd_model.fit(X_train, y_train)

-- Epoch 1
Norm: 3379011692498.85, NNZs: 13, Bias: 17684539633.540703, T: 354, Avg. loss: 199468532583827320826230734848.000000
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 1984678471469.27, NNZs: 13, Bias: 17833015552.240505, T: 708, Avg. loss: 93795310981835092141033914368.000000
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 1265074610458.99, NNZs: 13, Bias: 13874345818.993456, T: 1062, Avg. loss: 76004735538129981570189099008.000000
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 1118110170091.65, NNZs: 13, Bias: 6762286473.783154, T: 1416, Avg. loss: 58992220339907496703468503040.000000
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 1548299398973.92, NNZs: 13, Bias: 3623896539.959991, T: 1770, Avg. loss: 53733948208744075018857611264.000000
Total training time: 0.00 seconds.
-- Epoch 6
Norm: 988963787221.55, NNZs: 13, Bias: 6728708496.871233, T: 2124, Avg. loss: 51177833175292142338179072000.000000
Total training time: 0.00 seconds.
-- Epoch 7
Norm: 1658912432

SGDRegressor(verbose=1)

## 7. 예측(평가)

In [21]:
linear_model.score(X_train, y_train)

0.7611598772034454

In [22]:
linear_model.score(X_test, y_test)

0.6771696999851695

In [23]:
sgd_model.score(X_train, y_train)

-3.1525591224227085e+26

In [24]:
sgd_model.score(X_test, y_test)

-2.761062742872578e+26

### 특성확장
 - linear 모델의 성능이 굉장히 좋지 않게 나오고 있다
 - 이유는 특성의 숫자가 적기 때문 -> 해결하기 위해 특성확장(특성곱)을 해보자

In [43]:
col = X.columns
col

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT'],
      dtype='object')

In [45]:
# 1. 전체 컬럼을 순서대로 꺼내오는 for문
for i in range(col.size):
    # 2. 곱해지는 컬럼을 정하는 for문
    for j in range(i, col.size):
        # i * j = i * j
        X[col[i]+"*"+col[j]] = X[col[i]] * X[col[j]]

In [47]:
X.shape

(506, 104)

In [51]:
X.head(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,...,TAX*TAX,TAX*PTRATIO,TAX*B,TAX*LSTAT,PTRATIO*PTRATIO,PTRATIO*B,PTRATIO*LSTAT,B*B,B*LSTAT,LSTAT*LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,...,87616.0,4528.8,117482.40,1474.08,234.09,6072.570,76.194,157529.6100,1976.5620,24.8004
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,...,58564.0,4307.6,96049.80,2211.88,316.84,7064.820,162.692,157529.6100,3627.6660,83.5396
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,...,58564.0,4307.6,95064.86,975.26,316.84,6992.374,71.734,154315.4089,1583.1049,16.2409
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,...,49284.0,4151.4,87607.86,652.68,349.69,7379.581,54.978,155732.8369,1160.2122,8.6436
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,...,49284.0,4151.4,88111.80,1183.26,349.69,7422.030,99.671,157529.6100,2115.4770,28.4089


### 데이터 스케일링
- 주의사항 : 데이터 전처리 마지막에 사용
- 결측치가 없는 상태에서 사용해야 하고 이상치는 있어도 됨

In [54]:
from sklearn.preprocessing import MinMaxScaler

In [56]:
minmax = MinMaxScaler()

In [58]:
minmax.fit(X)   # 문제데이터인 X가 가지고 있는 범위로 학습

MinMaxScaler()

In [60]:
# 문제(X)의 기준으로 변환
X_trans = minmax.transform(X)
X_trans

array([[0.00000000e+00, 1.80000000e-01, 6.78152493e-02, ...,
        1.00000000e+00, 1.30834308e-01, 1.51574863e-02],
       [2.35922539e-04, 0.00000000e+00, 2.42302053e-01, ...,
        1.00000000e+00, 2.40435074e-01, 5.59846615e-02],
       [2.35697744e-04, 0.00000000e+00, 2.42302053e-01, ...,
        9.79596197e-01, 1.04716511e-01, 9.20813385e-03],
       ...,
       [6.11892474e-04, 0.00000000e+00, 4.20454545e-01, ...,
        1.00000000e+00, 1.48222891e-01, 2.00292898e-02],
       [1.16072990e-03, 0.00000000e+00, 4.20454545e-01, ...,
        9.82690814e-01, 1.68869818e-01, 2.71055404e-02],
       [4.61841693e-04, 0.00000000e+00, 4.20454545e-01, ...,
        1.00000000e+00, 2.07238689e-01, 4.10789948e-02]])

In [62]:
# 스케일링 된 X_trans 값으로 다시 분류
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_trans, y, 
                                                    test_size = 0.3, 
                                                    random_state = 5)

In [64]:
# 크기확인
print('문제 데이터 크기 :', X_train.shape, y_train.shape)
print('답 데이터 크기 :', X_test.shape, y_test.shape)

문제 데이터 크기 : (354, 104) (354,)
답 데이터 크기 : (152, 104) (152,)


In [66]:
linear_model.fit(X_train, y_train)
sgd_model.fit(X_train, y_train)

-- Epoch 1
Norm: 8.55, NNZs: 104, Bias: 3.246175, T: 354, Avg. loss: 43.823356
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 9.48, NNZs: 104, Bias: 3.763063, T: 708, Avg. loss: 26.783518
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 10.00, NNZs: 104, Bias: 4.056528, T: 1062, Avg. loss: 24.118312
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 10.51, NNZs: 104, Bias: 4.356280, T: 1416, Avg. loss: 21.847071
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 10.87, NNZs: 104, Bias: 4.527643, T: 1770, Avg. loss: 20.549263
Total training time: 0.00 seconds.
-- Epoch 6
Norm: 11.31, NNZs: 104, Bias: 4.777588, T: 2124, Avg. loss: 19.285335
Total training time: 0.00 seconds.
-- Epoch 7
Norm: 11.67, NNZs: 104, Bias: 4.969603, T: 2478, Avg. loss: 18.333521
Total training time: 0.00 seconds.
-- Epoch 8
Norm: 11.97, NNZs: 104, Bias: 5.103468, T: 2832, Avg. loss: 17.510286
Total training time: 0.00 seconds.
-- Epoch 9
Norm: 12.39, NNZs: 104, Bias: 5.336181, T: 3186, Avg. loss: 16.89

SGDRegressor(verbose=1)

In [68]:
linear_model.score(X_train, y_train)

0.933247168380447

In [70]:
linear_model.score(X_test, y_test)

0.8093536758919457

In [72]:
sgd_model.score(X_train, y_train)

0.7613621886770914

In [74]:
sgd_model.score(X_test, y_test)

0.683149473718696

### 규제를 적용시킨 모델
- LinearRegression 모델이 가지고 있는 단점
    - 모델이 잘못되었을 경우 수정이 안됨 -> 규제를 가해서 단점을 해소
    - LinearRegression + L1규제 => Lasso
    - LinearRegression + L2규제 => Ridge
- 규제? 선형모델에서 가중치(w)에 영향을 주겠다 -> 모델에 개입하겠다

In [77]:
from sklearn.linear_model import Lasso, Ridge

#### Ridge 활용

In [80]:
# 하이퍼파라미터(alpha) 튜닝하는 함수 정의
def ridge_alpha(alpha):
    # ridge모델 생성
    ridge = Ridge(alpha = alpha)
    # 학습
    ridge.fit(X_train, y_train)
    # 결과확인
    print("훈련세트 점수 :", ridge.score(X_train, y_train))
    print("테스트세트 점수 :", ridge.score(X_test, y_test))

In [88]:
ridge_alpha(100)
# alpha값의 기본값은 1
# alpha 값을 증가 ==> 규제를 늘리겠다. 과대적합일 때 사용 -> 과소적합에 걸릴 위험이 있다
# alpha 값을 감소 ==> 규제를 줄이겠다. 과소적합일 때 사용 -> 과대적합에 걸릴 위험이 있다
# ==> alpha값이 0이면 규제를 안한 것과 같다 ==> LinearRegression과 같다.
# ridge 모델은 규제를 하든 안하든 전체 가중치를 사용 -> 모든 컬럼을 사용
# ㄴ> 전체 컬럼이 고루고루 중요할 때 사용

훈련세트 점수 : 0.5867470593740554
테스트세트 점수 : 0.4908590961177438


#### Lasso 활용

In [91]:
import numpy as np

In [95]:
# 하이퍼파라미터(alpha) 튜닝하는 함수 정의
def lasso_alpha(alpha):
    # Lasso모델 생성
    lasso = Lasso(alpha = alpha)
    # 학습
    lasso.fit(X_train, y_train)
    # 결과확인
    print("훈련세트 점수 :", lasso.score(X_train, y_train))
    print("테스트세트 점수 :", lasso.score(X_test, y_test))
    # 사용한 특성의 수 확인
    print("사용한 특성의 수 :", np.sum(lasso.coef_ != 0))

In [105]:
lasso_alpha(0.0001)

# 규제를 늘릴수록 값이 잘 안나옴(과소적합)
# 규제를 줄일수록 값이 잘나옴(과대적합)
# 규제를 줄일수록 사용하는 컬럼의 개수가 늘어난다
# ㄴ> 컬럼의 개수를 모두 사용하면 LinearRegression 모델 사용했을 때와 거의 동일
# Lasso모델은 특정 특성이 중요하고 나머지 특성들의 중요도가 현저히 낮을 때 사용

훈련세트 점수 : 0.9145337059961222
테스트세트 점수 : 0.8344026382098213
사용한 특성의 수 : 103
